In [1]:
#importing required packages

import requests
import scrapy
from bs4 import BeautifulSoup
from scrapy import Selector
from scrapy.crawler import CrawlerProcess
from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner
import string
import pandas as pd

ModuleNotFoundError: No module named 'scrapy'

In [ ]:
#url of the website to scrape

url='https://internshala.com/fresher-jobs'

In [ ]:
#raw html data from website

html=requests.get(url).content
#html

In [ ]:
#html in well formatted form

pretty_html=BeautifulSoup(html).prettify()
#pretty_html

In [ ]:
#creating scrapy's selector object to select the html above

sel=Selector(text=pretty_html)

In [ ]:
#final, the dictionary which would be later converted to a dataframe, final_df, to store the scraped data

final={'ID':[],'Job_Post':[],'Company':[],'Location':[],'CTC':[]}
final_df=pd.DataFrame()

In [ ]:
#scraping available job's ID

final['ID']=sel.xpath('//div[@class="container-fluid individual_internship visibilityTrackerItem"]/@internshipid').extract()
final['ID']

In [ ]:
#scraping available job's name

final['Job_Post']=sel.xpath('//div[@class="heading_4_5 profile"]/a').xpath('normalize-space(./text())').extract()
final['Job_Post']

In [ ]:
final['Company']=sel.xpath('//a[@class="link_display_like_text"]').xpath('normalize-space(./text())').extract()
final['Company']

In [ ]:
#scraping the city name for the job (the toughest part, due to the fact that a single job post can have mutiple job locations)

#scraping each job's locations in a different list thus forming a nested list,ind_location
ind_location=[]
final['Location']=[]
for i in final['ID']:
    ind_location.append(sel.xpath("//div[@internshipid='" + i + "']//a[@class='location_link']").xpath('normalize-space(./text())').extract())

#converting the nested list above and storing in a normal list, final['Location']  
for i in ind_location:
    s=''
    c=0
    if len(i)>1:
        for j in i:
            if c>0:
                s=s+', '+j
            else:
                s=s+j
            c+=1
    else:
        s=i[0]
    final['Location'].append(s)

final['Location']

In [ ]:
to_remove={0:["\n                ","\n               "],\
           1:['\n             ','\n            ']}

def remove_(attr,remove):
    for i in remove:
        final[attr]=[f.replace(i, "") for f in final[attr]]

final['CTC']=sel.xpath('//div[@class="internship_other_details_container"]/div[1]/div[2]/div[2]/text()').extract()                                   
remove_('CTC',to_remove[0])
final['CTC'] = ['₹'+v for i, v in enumerate(final['CTC']) if(i % 2)]
final['CTC']

In [ ]:
final['Apply By']=sel.xpath('//div[@class="internship_other_details_container"]/div[@class="other_detail_item_row"][2]//div[@class="item_body"]').xpath('normalize-space(./text())').extract()
final['Apply By']

In [ ]:
final_df=pd.DataFrame(final)
final_df